# Emergency Department Environment

In [2]:
import simpy
import random
import numpy as np

In [66]:
class Project:
    def __init__(self, name, time_distribution, staff_needed, **kwargs):
        self.name = name
        self.time_distribution = time_distribution
        self.staff_needed = staff_needed  # 该项目需要的工作人员数量
        if time_distribution == "uniform":
            self.min_time = kwargs.get('min_time', 0)
            self.max_time = kwargs.get('max_time', 0)
        elif time_distribution == "triangular":
            self.min_time = kwargs.get('min_time', 0)
            self.mode_time = kwargs.get('mode_time', 0)
            self.max_time = kwargs.get('max_time', 0)
    
    def get_time(self):
        if self.time_distribution == "uniform":
            return random.randint(self.min_time, self.max_time)
        elif self.time_distribution == "triangular":
            return int(np.random.triangular(self.min_time, self.mode_time, self.max_time))

In [67]:
class Staff:
    def __init__(self):
        self.staff_dict = {}  # 存储各种类型的工作人员和其数量
        self.staff_prices = {}  # 存储各种类型的工作人员和其价格

    def add_staff(self, staff_type, quantity, price):
        self.staff_dict[staff_type] = quantity
        self.staff_prices[staff_type] = price

    def assign_staff(self, staff_type, quantity):
        self.staff_dict[staff_type] -= quantity

    def release_staff(self, staff_type, quantity):
        self.staff_dict[staff_type] += quantity


In [182]:
class Patient:
    def __init__(self, env, name, projects, project_probabilities):
        self.env = env
        self.name = name
        self.all_projects = projects
        self.project_probabilities = project_probabilities
        
        self.current_project = None
        self.current_project_start_time = 0
        self.current_project_end_time = 0
        self.current_project_time = 0
        
        self.in_project = False
        self.is_waiting = False
        
        self.waiting_time = 0
        self.emergency_room_waiting_time = 0
        self.total_time = 0  # 记录总共花费的时间
        
        self.start_times = {}  # 记录每个项目的开始时间
        self.end_times = {}    # 记录每个项目的结束时间
        self.project_times = {}  # 记录每个项目使用的时间

        self.completed_projects = []  # 记录完成的项目

    def execute_project(self, project):
        self.in_project = True

        self.current_project_start_time = self.env.now
        self.start_times[self.current_project.name] = self.current_project_start_time  # 记录项目开始时间
        project_time = self.current_project.get_time()
        self.current_project_time = project_time
        # end_time = self.env.now
        # self.end_times[self.current_project.name] = end_time  # 记录项目结束时间
        self.project_times[self.current_project.name] = project_time  # 记录项目使用的时间
        self.total_time += project_time  # 更新总共花费的时间
        self.completed_projects.append(project)  # 记录完成的项目
        
    def check_finish(self):
        if self.env.now - self.current_project_start_time > self.current_project_time:
            self.in_project = False
            self.current_project_end_time = self.env.now
            if  self.current_project != None:
                self.end_times[self.current_project.name] = self.current_project_end_time

    def choose_next_project(self):
        current_project_name = self.current_project.name if self.current_project else None
        next_project_probabilities = self.project_probabilities.get(current_project_name, self.project_probabilities["default"])
        projects = list(next_project_probabilities.keys())
        probabilities = list(next_project_probabilities.values())
        # print(projects,probabilities)
        chosen_project = random.choices(projects, probabilities)[0]
        chosen_project = [project for project in self.all_projects if project.name == chosen_project][0]
        self.current_project = chosen_project
        return chosen_project


In [183]:
class EmergencyDepartment:
    def __init__(self, env, staff, projects, project_probabilities, arrival_time):
        self.env = env
        self.staff = staff
        self.projects = projects
        self.project_probabilities = project_probabilities
        self.patients = []
        self.discharged_patient = []
        self.arrival_time = arrival_time

    def admit_patient(self, patient):
        self.patients.append(patient)
        
    def process_patient(self, patients):
        while True:
            if self.env.now in self.arrival_time:
                self.admit_patient(Patient(self.env, len(self.patients)+1, self.projects, self.project_probabilities))
            for patient in patients:
                if patient in self.discharged_patient:
                    continue  # 跳过已经出院的患者
                
                if patient.current_project != None:
                    if patient.current_project.name == "Discharge":
                        self.discharged_patient.append(patient)
                        continue

                if not patient.in_project and not patient.is_waiting:  # 如果当前没有项目，则选择下一个项目
                    project = patient.choose_next_project()
                    if self.can_start_project(project):
                        self.assign_staff(project)
                        patient.execute_project(project)
                    else:
                        patient.waiting_time += 1
                        patient.is_waiting = True      
                    continue
                
                if not patient.in_project and patient.is_waiting:  # 如果当前没有项目，则选择下一个项目
                    project = patient.current_project
                    
                    if self.can_start_project(project):
                        self.assign_staff(project)
                        patient.execute_project(project)
                        patient.is_waiting = False
                    else:
                        patient.waiting_time += 1    
                    continue
                
                patient.check_finish()
                if not patient.in_project:
                    project = patient.completed_projects[-1]
                    self.release_staff(project)
                    
            yield self.env.timeout(1)  # 继续等待1个单位时间
            
    def can_start_project(self, project):
        staff_needed = project.staff_needed
        project_name = project.name
        if project_name in self.staff.staff_dict:
            available_staff = self.staff.staff_dict[project_name]
            return staff_needed <= available_staff
        else:
            return False

    def assign_staff(self, project):
        staff_needed = project.staff_needed
        project_name = project.name
        self.staff.assign_staff(project_name, staff_needed)

    def release_staff(self, project):
        staff_needed = project.staff_needed
        project_name = project.name
        self.staff.release_staff(project_name, staff_needed)


In [208]:
def calculate_average_waiting_time(waiting_times):
    if waiting_times:
        return sum(waiting_times) / len(waiting_times)
    else:
        return 0
    
def arrival_times(arrival_rates: list):
    hour = range(len(arrival_rates))
    arr_time = []
    for h in hour:
        for t in np.linspace(h*60,(h+1)*60,int(round(arrival_rates[h]))+1):
            arr_time.append(round(t))
        arr_time = arr_time[:-1]
    return arr_time


In [209]:
if __name__ == "__main__":
    env = simpy.Environment()
    
    # 定义病人每小时到达率
    arrival_rates = [5.33, 5.33, 3.82, 3.82, 3.06, 3.06, 4.54, 4.54, 6.81, 6.84, 8.32, 8.32, 
                     9.01, 9.01, 8.36, 7.5, 8.32, 7.5, 8.32, 8.32, 7.5, 5.33, 3.82, 2.3]
    # arrival_rates = [i*10 for i in arrival_rates]
    arrival_time = arrival_times(arrival_rates)
    
    # 创建不同项目
    projects = [
        Project("default", "uniform", staff_needed=0, min_time=0, max_time=0),
        Project("Reception", "uniform", staff_needed=1, min_time=5, max_time=10),
        Project("Examination room", "uniform", staff_needed=1, min_time=10, max_time=20),
        Project("Lab tests", "triangular", staff_needed=1, min_time=10, mode_time=20, max_time=30),
        Project("Reexamination process", "uniform", staff_needed=1, min_time=7, max_time=12),
        Project("Treatment room", "uniform", staff_needed=1, min_time=20, max_time=30),
        Project("Emergency room", "uniform", staff_needed=1, min_time=60, max_time=120),
        Project("Discharge", "uniform", staff_needed=0, min_time=0, max_time=0)  # 出院项目
    ]
    
    # 创建工作人员
    staff = Staff()
    staff.add_staff("default", 0, 0)
    staff.add_staff("Reception", 1, 0.4)
    staff.add_staff("Lab tests", 2, 0.5)
    staff.add_staff("Examination room", 2, 1.2)
    staff.add_staff("Reexamination process", 1, 1.2)
    staff.add_staff("Treatment room", 2, 0.3)
    staff.add_staff("Emergency room", 1, 0.3)
    staff.add_staff("Discharge", 0, 0)
    
    project_probabilities = {
        "default": {"Reception": 1.0, "Lab tests": 0.0, "Examination room": 0.0,
                    "Reexamination process": 0.0, "Treatment room": 0.0, "Emergency room": 0.0, "Discharge": 0.0},
        "Reception": {"Examination room": 1.0, "Lab tests": 0.0, "Reexamination process": 0.0,
                    "Treatment room": 0.0, "Emergency room": 0.0, "Discharge": 0.0},
        "Examination room": {"Treatment room": 0.2, "Lab tests": 0.5, "Discharge": 0.05, "Emergency room": 0.25,
                            "Reception": 0.0, "Reexamination process": 0.0},
        "Lab tests": {"Reexamination process": 1.0, "Reception": 0.0, "Examination room": 0.0,
                    "Treatment room": 0.0, "Emergency room": 0.0, "Discharge": 0.0},
        "Reexamination process": {"Treatment room": 0.4, "Discharge": 0.1, "Emergency room": 0.5,
                                "Reception": 0.0, "Lab tests": 0.0, "Examination room": 0.0},
        "Treatment room": {"Discharge": 1.0, "Reception": 0.0, "Lab tests": 0.0, "Examination room": 0.0,
                            "Reexamination process": 0.0, "Emergency room": 0.0},
        "Emergency room": {"Discharge": 1.0, "Reception": 0.0, "Lab tests": 0.0, "Examination room": 0.0,
                            "Reexamination process": 0.0, "Treatment room": 0.0},
        "Discharge": {"Reception": 0.0, "Lab tests": 0.0, "Examination room": 0.0,
                    "Reexamination process": 0.0, "Treatment room": 0.0, "Emergency room": 0.0}
    }

    emergency_department = EmergencyDepartment(env, staff, projects, project_probabilities, arrival_time)
    env.process(emergency_department.process_patient(emergency_department.patients))
    env.run(until=1440)  # 模拟24小时
    for patient in emergency_department.patients:
        print(f"Patient: {patient.name}")
        print(f"Project: {[project.name for project in patient.completed_projects]}")
        for project in projects[1:-1]:
            start_time = patient.start_times.get(project.name, "Not Started")
            end_time = patient.end_times.get(project.name, "Not Finished")
            project_time = patient.project_times.get(project.name, 0)
            print(f"Project: {project.name}, Start Time: {start_time}, End Time: {end_time}, Project Time: {project_time}")
        print(f"Total Time Spent: {patient.total_time}")
        print(f"Waiting Time: {patient.waiting_time}")
        print(f"emergency_room Waiting Time: {patient.emergency_room_waiting_time}")
        print("\n")
    waiting_times = [patient.waiting_time for patient in emergency_department.patients]
    average_waiting_time = calculate_average_waiting_time(waiting_times)
    print(f"所有病人的等待时间的平均值为: {average_waiting_time:.2f} 分钟")



Patient: 1
Project: ['Reception', 'Examination room', 'Lab tests', 'Reexamination process', 'Treatment room', 'Discharge']
Project: Reception, Start Time: 0, End Time: 10, Project Time: 9
Project: Examination room, Start Time: 11, End Time: 30, Project Time: 18
Project: Lab tests, Start Time: 31, End Time: 54, Project Time: 22
Project: Reexamination process, Start Time: 55, End Time: 67, Project Time: 11
Project: Treatment room, Start Time: 68, End Time: 97, Project Time: 28
Project: Emergency room, Start Time: Not Started, End Time: Not Finished, Project Time: 0
Total Time Spent: 88
Waiting Time: 0
emergency_room Waiting Time: 0


Patient: 2
Project: ['Reception', 'Examination room', 'Discharge']
Project: Reception, Start Time: 12, End Time: 23, Project Time: 10
Project: Examination room, Start Time: 24, End Time: 40, Project Time: 15
Project: Lab tests, Start Time: Not Started, End Time: Not Finished, Project Time: 0
Project: Reexamination process, Start Time: Not Started, End Time: 

In [210]:
# TODO 计算每小时病人出院率
# TODO 记录ER的等待时间
# TODO 检查代码逻辑
# TODO 写README
# TODO 可视化真实医院
# TODO 在符合等待时间的约束下使用回溯算法寻找可行域
# TODO 蒙特卡洛搜索寻找最优配置